Imports

In [10]:
import os
import glob
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from typing import List, Optional
from dataclasses import dataclass
from plotly.graph_objs import Figure

Constants

In [11]:
PMC_CONVENTIONAL = 4362.47
PMC_OPERATIVE_69 = 2024.481
PMC_OPERATIVE_138 = 2513.486

PQ_QUANTITY = 49

YEARS = [2026, 2028, 2030, 2032, 2034, 2036]
PENETRATION_LEVELS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Classes and functions

In [12]:
from dataclasses import dataclass
import pandas as pd

@dataclass
class BarData:
    '''
    A dataclass that retains relevant information about system's bars.

    This dataclass stores information about system bars, including the file name, name, voltage class, and a DataFrame.

    Attributes:
        _file_name (str): The file name associated with the bar data.

    Properties:
        file_name (str): The file name associated with the bar data.
        name (str): The name extracted from the file name (excluding extension).
        voltage_class (str): The voltage class extracted from the file name.
        df (pd.DataFrame): A DataFrame containing the bar data.
    '''

    _file_name: str

    @property
    def file_name(self) -> str:
        '''str: The file name associated with the bar data.'''
        return self._file_name

    @property
    def name(self) -> str:
        '''str: The name extracted from the file name (excluding extension).'''
        return self._file_name[9:-4]

    @property
    def voltage_class(self) -> str:
        '''str: The voltage class extracted from the file name.'''
        return self.name[-3:]

    @property
    def df(self) -> pd.DataFrame:
        '''pd.DataFrame: A DataFrame containing the bar data.'''
        return self.read_csv(self._file_name)
    
    @property
    def power_threshold(self) -> float:
        '''
        Get the power threshold for operational voltage based on a filtered DataFrame.

        Returns:
            float: Power threshold for operational voltage.
        '''
        filtered_df = self.df.loc[self.df[1].between(0.95, 0.96)] 
        
        if not filtered_df.empty:
            voltage_threshold_index = filtered_df[1].idxmin()
            power_threshold = filtered_df.at[voltage_threshold_index, 0]
        else:
            power_threshold = PMC_CONVENTIONAL

        return power_threshold
    
    def read_csv(self, _file_name: str) -> pd.DataFrame:
        '''
        Read bar data from a CSV file and return it as a DataFrame.

        Args:
            _file_name (str): The file name of the CSV file.

        Returns:
            pd.DataFrame: A DataFrame containing the bar data.
        '''
        df = pd.read_csv(_file_name, delimiter= ';', header= None)
        return df
    
    def calculate_operational_vsm(self, operational_point_power: float) -> float:
        '''
        Calculate the Voltage Stability Margin (VSM) based on the operational point power.

        Parameters:
            operational_point_power (float): The operational point power.

        Returns:
            float: The calculated Voltage Stability Margin (VSM).
        '''
        return 100 * ((self.power_threshold - operational_point_power) / self.power_threshold)

    def __repr__(self) -> str:
        '''str: A string representation of the BarData instance.'''
        return '{} -- Voltage class: {} -- Power Threshold for Operational Voltage: {:.2f}'.format(self.name.strip(),
                                                                                                   self.voltage_class,
                                                                                                   self.power_threshold)
    
def get_random_sum(loads: List[int],
                   get_amount: int,
                   iterations: int
                   ) -> int:
    '''
    Calculate the sum of a random selection of load values in megawatts (MW).

    This function randomly selects a specified number of load values from the provided list and calculates their sum in megawatts (MW).

    Args:
        loads (List[int]): A list of load values in megawatts (MW).
        get_amount (int): The number of load values to randomly select and sum.
        times (int): The amount of times the function will be repeated.

    Returns:
        int: The sum of the randomly selected load values in megawatts (MW).
    '''
    total_sum = 0

    for _ in range(iterations):
        sampled_loads = random.sample(loads, get_amount)
        total_sum += sum(sampled_loads)

    return total_sum / 1000

def increment_power(load: float,
                    increment_amount: int,
                    increment_percentage: float
                    ) -> List[float]:
    '''
    Increment a load value by a specified number and percentage multiple times.

    This function takes a starting load value and increments it by a specified number and percentage for a given number of times.

    Args:
        load (float): The initial load value.
        increment_amount (int): The number of times to increment the load.
        increment_percentage (float): The percentage by which to increment the load in each step.

    Returns:
        list: A list containing the incremented load values, including the initial load.
    '''
    incremented_values = [load]

    for _ in range(increment_amount):
        load *= (1 + increment_percentage)
        incremented_values.append(load)

    return incremented_values

def make_matrices(max_power: float,
                  load_value: List[int],
                  charging_station_power: List[float],
                  penetration_level: int,
                  increment: bool = False
                  ) -> tuple[List[List], List[List]]:
    '''
    Generate matrices of power and margin of stability based on max power, load, charging station power, and increment settings.

    This function calculates power and margin of stability matrices for a given load value and power value reference, considering various charging station power scenarios.

    Args
        max_power (float): Reference power value.
        load_value (float): The base load value.
        charging_station_power (list): A list of charging station power values.
        increment (bool, optional): Whether to increment load with charging station power scenarios (default is False).

    Returns:
        tuple: A tuple containing two lists of lists:
            - A power matrix: List of lists representing power values for different scenarios.
            - A margin of stability matrix: List of lists representing margin of stability values for different scenarios.
    '''
    pow_matrix = []
    vsm_matrix = []
    evh_matrix = []

    for i in range(PQ_QUANTITY):
        pow_row = []
        vsm_row = []
        evh_row = []

        for _ in range(200):
            charging_site_power = (i + 1) * get_random_sum(charging_station_power,
                                                           get_amount= 3,
                                                           iterations= penetration_level)
            
            power_value = (load_value + charging_site_power) if increment else load_value

            vsm_value = 100.0 * ((max_power - power_value) / max_power)

            pow_row.append(power_value)
            vsm_row.append(vsm_value)
            evh_row.append(charging_site_power)

        pow_matrix.append(pow_row)
        vsm_matrix.append(vsm_row)
        evh_matrix.append(evh_row)
    
    return pow_matrix, vsm_matrix

def filter_by_voltage(bars: List[BarData],
                      voltage_class: Optional[str] = None
                      ) -> List[BarData]:
    '''
    Filter a list of BarData instances by their voltage_class attribute.

    Args:
        bars (List[BarData]): List of BarData instances to filter.
        voltage_class (str): The voltage class to filter by. Defaults to None.

    Returns:
        List[BarData]: List of filtered BarData instances.
    '''

    if voltage_class is None:
        return bars
    
    filtered_bars = [bar for bar in bars if bar.voltage_class == voltage_class]
    
    return filtered_bars

def matrix_to_list(target_matrix: List[List[float]]) -> List[float]:
    '''
    Turns a matrix into a list.

    Args:
        target_matrix (List[List[float]]): Matrix to be turned into a list.

    Returns:
        List[float]: List of elements of target_matrix.
    '''

    matrix_as_list = [element for row in target_matrix for element in row]

    return matrix_as_list

def save_as_csv(target_data, file_name, folder_path):
    '''
    Save a list of data to a CSV file.

    Args:
        target_data (list): The data to be saved.
        file_name (str): The name of the CSV file.
        folder_path (str): The path to the folder where the file will be saved.

    Returns:
        None
    '''
    file_path = os.path.join(folder_path, file_name)

    pd.DataFrame(target_data).to_csv(file_path, index= False, header= None)

def avg(input_list: List) -> float:
    average_value = sum(input_list) / len(input_list) 
    
    return average_value 

def conventional_heatmap(target_matrix: dict,
                         title: str) -> Figure:
    '''
    Create a heatmap using Plotly Express.

    Parameters:
        target_matrix (dict): A nested dictionary containing matrix data.
        title (str): A string to be used as title.

    Returns:
        A Figure object.
    '''
    min_values_matrix = {}
    avg_values_matrix = {}
    max_values_matrix = {}

    for year in YEARS:
        min_values_matrix[year] = {}
        avg_values_matrix[year] = {}
        max_values_matrix[year] = {}

        for penetration_level in PENETRATION_LEVELS:
            current_matrix = target_matrix[year][penetration_level]
            
            min_values_matrix[year][penetration_level] = min_value = min(min(current_matrix))
            avg_values_matrix[year][penetration_level] = avg_value = avg(matrix_to_list(current_matrix))
            max_values_matrix[year][penetration_level] = max_value = max(max(current_matrix))

    min_df = pd.DataFrame([(year, penetration_level, min_value) 
                    for year, penetration_levels_dict in min_values_matrix.items()
                    for penetration_level, min_value in penetration_levels_dict.items()],
                    columns=['Years', 'Penetration Levels', 'Avg Values'])

    fig = px.imshow(min_df.pivot(index= 'Penetration Levels',
                                 columns= 'Years',
                                 values= 'Avg Values'),
                    x= YEARS,
                    y= PENETRATION_LEVELS[::-1],
                    labels= dict(color= 'Avg Values'),
                    title= title,
                    color_continuous_scale= 'Portland',
                    aspect= 'auto')
    
    fig.update_layout(title= title,
                      xaxis_title= 'Year',
                      yaxis_title= 'Pen Level')

    fig.update_layout(yaxis=dict(tickvals= list(range(0, 11)),
                                 ticktext= list(range(10, -1, -1))))
    fig.update_layout(xaxis=dict(tickvals= YEARS,
                                 ticktext= YEARS))

    for year in YEARS:
        for idx, penetration_level in enumerate(PENETRATION_LEVELS[::-1]):
            min_value = min_values_matrix[year][penetration_level]
            max_value = max_values_matrix[year][penetration_level]
            fig.add_annotation(x=year, y=idx, text='{:.2f}<br>{:.2f}'.format(max_value, min_value),
                            showarrow=False, font=dict(color='white'), align='center')

    fig.update_layout(height= 600,
                      width= 1200)
    fig.update_layout(title_x= 0.5)

    return fig
    
def bar_group_heatmap(target_directory: str,
                            title: str
                            ) -> Figure:
    '''
    Generate a heatmap from CSV files in the specified directory.

    Parameters:
    - target_directory (str): The path to the directory containing CSV files.
    - title (str): The title of the heatmap.

    Returns:
    - Figure: Plotly figure object representing the heatmap.
    '''

    min_values_dict = {}
    max_values_dict = {}

    for desired_year in YEARS:
        pattern = f'*_{desired_year}_*.csv'
        files = glob.glob(os.path.join(target_directory, pattern))

        for file_path in files:
            df = pd.read_csv(file_path)

            _, barname, year_from_filename, _, _, pen_level_with_extension = os.path.basename(file_path).split('_')

            year_from_filename = int(year_from_filename)
            pen_level = int(pen_level_with_extension.split('.')[0])

            if year_from_filename not in min_values_dict:
                min_values_dict[year_from_filename] = {}
                max_values_dict[year_from_filename] = {}

            min_values_dict[year_from_filename][pen_level] = df.min().min()
            max_values_dict[year_from_filename][pen_level] = df.max().max()

            for year, values in min_values_dict.items():
                sorted_pen_levels = sorted(values,
                                           key= values.get,
                                           reverse= True)
                min_values_dict[year] = {pen_level: values[pen_level] for pen_level in sorted_pen_levels}
            
            for year, values in max_values_dict.items():
                sorted_pen_levels = sorted(values,
                                           key= values.get,
                                           reverse= True)
                max_values_dict[year] = {pen_level: values[pen_level] for pen_level in sorted_pen_levels}

    min_df = pd.DataFrame(min_values_dict)

    fig = px.imshow(min_df,
                    labels=dict(x="Pen Level", y="Year", color="Minimum Value"),
                    x=min_df.columns,
                    y=min_df.index[::-1],
                    color_continuous_scale="Portland",
                    aspect= 'auto')

    fig.update_layout(title= title,
                      xaxis_title= 'Year',
                      yaxis_title= 'Pen Level')

    fig.update_layout(yaxis= dict(tickvals= list(range(0, 11)),
                                  ticktext= list(range(10, -1, -1))))
    fig.update_layout(xaxis= dict(tickvals= YEARS,
                                  ticktext= YEARS))

    for year in YEARS:
        for idx, penetration_level in enumerate(PENETRATION_LEVELS[::-1]):
            min_value = min_values_dict[year][penetration_level]
            max_value = max_values_dict[year][penetration_level]
            fig.add_annotation(x=year, y=idx, text='{:.2f}<br>{:.2f}'.format(max_value, min_value),
                            showarrow=False, font=dict(color='white'), align='center')
            
    fig.update_layout(height= 600,
                      width= 1200)
    fig.update_layout(title_x= 0.5)

    return fig

def housing_capacity_heatmap(target_directory: str,
                             title: str
                            ) -> Figure:
    '''
    Generate a heatmap from CSV files in the specified directory.

    Parameters:
    - target_directory (str): The path to the directory containing CSV files.
    - title (str): The title of the heatmap.

    Returns:
    - Figure: Plotly figure object representing the heatmap.
    '''

    min_values_dict = {}
    avg_values_dict = {}
    max_values_dict = {}

    for desired_year in YEARS:
            pattern = f'*_{desired_year}_*.csv'
            files = glob.glob(os.path.join(target_directory, pattern))

            for file_path in files:
                avaible_power_df = pd.read_csv(file_path)

                _, _, _, year_from_filename, _, _, pen_level_with_extension = os.path.basename(file_path).split('_')

                year_from_filename = int(year_from_filename)
                pen_level = int(pen_level_with_extension.split('.')[0])

                if year_from_filename not in min_values_dict:
                    min_values_dict[year_from_filename] = {}
                    avg_values_dict[year_from_filename] = {}
                    max_values_dict[year_from_filename] = {}

                min_values_dict[year_from_filename][pen_level] = int(round(avaible_power_df // WORST_CASE_STATION_POWER).min().min())
                avg_values_dict[year_from_filename][pen_level] = int(round(avaible_power_df // AVG_CASE_STATION_POWER).mean().mean())
                max_values_dict[year_from_filename][pen_level] = int(round(avaible_power_df // BEST_CASE_STATION_POWER).max().max())

                for year, values in min_values_dict.items():
                    sorted_pen_levels = sorted(values,
                                               key= values.get,
                                               reverse= True)
                    min_values_dict[year] = {pen_level: values[pen_level] for pen_level in sorted_pen_levels}
                
                for year, values in max_values_dict.items():
                    sorted_pen_levels = sorted(values,
                                               key= values.get,
                                               reverse= True)
                    max_values_dict[year] = {pen_level: values[pen_level] for pen_level in sorted_pen_levels}

    avg_df = pd.DataFrame(avg_values_dict).sort_index(ascending= True)

    fig = px.imshow(avg_df,
                    labels= dict(x= "Year", 
                                 y= "Pen Level", 
                                 color= "Avg Value"),
                    x= avg_df.columns,
                    y= avg_df.index[::-1],
                    color_continuous_scale= "Portland",
                    aspect= 'auto')

    fig.update_layout(title= title,
                      xaxis_title= 'Year',
                      yaxis_title= 'Pen Level')

    fig.update_layout(yaxis= dict(tickvals= list(range(0, 11)),
                                  ticktext=list(range(10, -1, -1))))
    fig.update_layout(xaxis= dict(tickvals=YEARS,
                                  ticktext=YEARS))

    for year in YEARS:
            for idx, penetration_level in enumerate(PENETRATION_LEVELS[::-1]):
                min_value = min_values_dict[year][penetration_level]
                max_value = max_values_dict[year][penetration_level]
                fig.add_annotation(x= year,
                                   y= idx,
                                   text= '{}<br>{}'.format(max_value, min_value),
                                   showarrow= False,
                                   font= dict(color='white'),
                                   align= 'center')
                
    fig.update_layout(height= 600,
                      width= 1200)
    fig.update_layout(title_x= 0.5)


    return fig


BarData objects

In [13]:
folder_path = r'PVs'

bars = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        
        bar = BarData(file_path)

        bars.append(bar)

Load defintions

In [ ]:
HEAVY_LOAD = 1642.69
CHARGING_STATION_POWER = [114, 147, 180, 213]

WORST_CASE_STATION_POWER = CHARGING_STATION_POWER[-1] * 3 / 1000
AVG_CASE_STATION_POWER = avg(CHARGING_STATION_POWER) * 3 / 1000
BEST_CASE_STATION_POWER = CHARGING_STATION_POWER[0] * 3 / 1000

incremented_loads = increment_power(HEAVY_LOAD, 5, 0.082)

Conventional VSM

In [ ]:
pow_matrix = {}
vsm_matrix = {}

for year in YEARS:

    load_data = incremented_loads[YEARS.index(year)]

    pow_matrix[year] = {}
    vsm_matrix[year] = {}
    
    for penetration_level in PENETRATION_LEVELS:
        pow_matrix[year][penetration_level], \
        vsm_matrix[year][penetration_level] = make_matrices(max_power= PMC_CONVENTIONAL,
                                                            load_value= load_data,
                                                            charging_station_power= CHARGING_STATION_POWER,
                                                            penetration_level= penetration_level,
                                                            increment= True
                                                            )

        save_as_csv(target_data= vsm_matrix[year][penetration_level],
                    file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}.csv',
                    folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Conventional_VSMs')
        
        save_as_csv(target_data= pow_matrix[year][penetration_level],
                    file_name= f'pow_matrix_{year}_pen_level_{penetration_level}.csv',
                    folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Power_for_conventional_VSMs')
        

Operative 69kV reference VSM

In [ ]:
# pow_matrix_69_reference = {}
# vsm_matrix_69_reference = {}

# for year in YEARS:

#     load_data = incremented_loads[YEARS.index(year)]

#     pow_matrix_69_reference[year] = {}
#     vsm_matrix_69_reference[year] = {}
    
#     for penetration_level in PENETRATION_LEVELS:
#         pow_matrix_69_reference[year][penetration_level], \
#         vsm_matrix_69_reference[year][penetration_level], = make_matrices(max_power= PMC_OPERATIVE_69,
#                                                                   load_value= load_data,
#                                                                   charging_station_power= charging_station_power,
#                                                                   penetration_level= penetration_level,
#                                                                   increment= True)

#         save_as_csv(target_data= vsm_matrix_69_reference[year][penetration_level],
#                     file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}_69_op.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\69_operative_VSMs')

Operative 138kV reference VSM

In [ ]:
# pow_matrix_138_reference = {}
# vsm_matrix_138_reference = {}

# for year in YEARS:

#     load_data = incremented_loads[YEARS.index(year)]

#     pow_matrix_138_reference[year] = {}
#     vsm_matrix_138_reference[year] = {}
    
#     for penetration_level in PENETRATION_LEVELS:
#         pow_matrix_138_reference[year][penetration_level], \
#         vsm_matrix_138_reference[year][penetration_level], = make_matrices(max_power= PMC_OPERATIVE_138,
#                                                                   load_value= load_data,
#                                                                   charging_station_power= charging_station_power,
#                                                                   penetration_level= penetration_level,
#                                                                   increment= True)

#         save_as_csv(target_data= vsm_matrix_138_reference[year][penetration_level],
#                     file_name= f'vsm_matrix_{year}_pen_level_{penetration_level}_138_op.csv',
#                     folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\138_operative_VSMs')

Plots

In [ ]:
fig = go.Figure()

voltage_to_be_filtered = None

filtered_bars = filter_by_voltage(bars, voltage_to_be_filtered)

for index, bar in enumerate(filtered_bars):
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'Bar {index + 1}: {bar.name}')
    )

fig.update_layout(
    xaxis_title= 'Load (MW)',
    yaxis_title= 'Voltage (pu)',
    title= f'PxV curves for all voltage levels' \
            if voltage_to_be_filtered == None \
            else f'PxV curves for {voltage_to_be_filtered}kV voltage level',
    title_x= 0.5
)

fig.update_layout(height= 600,
                  width= 1200)


fig.show()


In [ ]:
# fig = go.Figure()

# vsm_per_year = []
# vsm_per_year_69 = []
# vsm_per_year_138 = []

# for year in YEARS:
#     vsm_per_year.append(vsm_matrix[year][0][0][0])
#     vsm_per_year_69.append(vsm_matrix_69_reference[year][0][0][0])
#     vsm_per_year_138.append(vsm_matrix_138_reference[year][0][0][0])

# fig.add_trace(
#     go.Scatter(x= YEARS,
#                y= vsm_per_year,
#                name= 'Conventional VSM')
# )

# fig.add_trace(
#     go.Scatter(x= YEARS,
#                y= vsm_per_year_69,
#                name= '69kV operative VSM')
# )

# fig.add_trace(
#     go.Scatter(x= YEARS,
#                y= vsm_per_year_138,
#                name= '138kV operative VSM')
# )

# fig.update_layout(
#     xaxis_title= 'Years',
#     yaxis_title= 'Voltage margin (%)',
#     title= 'Voltage Stability Margin Evolution Over the Years',
#     title_x= 0.5
# )

# fig.show()

In [ ]:
fig1 = conventional_heatmap(target_matrix= vsm_matrix,
                    title= 'Conventional Voltage Stability Margin Evolution <br> with Penetration Levels Over the Years')

fig1.show()

# fig2 = make_heatmap(target_matrix= vsm_matrix_69_reference,
#                     title= '69kV Operative Point Maintenance Voltage Stability Margin <br> Evolution with Penetration Levels Over the Years')

# fig2.show()                                     

# fig3 = make_heatmap(target_matri x= vsm_matrix_138_reference,
#                     title= '138kV Operative Point Maintenance Voltage Stability Margin <br> Evolution with Penetration Levels Over the Years')

# fig3.show()                                     

In [ ]:
# # DATA COLLECTION DONE!!!

# target_voltage = '138' 
# filtered_bars = filter_by_voltage(bars, target_voltage)

# def calculate_vsm(bar, power_level):
#     return bar.calculate_operational_vsm(power_level)

# for bar in filtered_bars:
#     for year in YEARS:
#         for penetration_level in PENETRATION_LEVELS:
#             vsm_values = pd.DataFrame(pow_matrix[year][penetration_level]).apply(lambda x: calculate_vsm(bar, x), axis= 1)

#             save_as_csv(target_data= vsm_values,
#                         file_name= f'{bar.name}_year_{year}_pen_level_{penetration_level}.csv'.strip(),
#                         folder_path= rf'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Operative_VSMs_by_bar_{target_voltage}')


In [ ]:
# # DATA COLLECTION DONE!!!

# target_voltage = '069' 
# filtered_bars = filter_by_voltage(bars, target_voltage)

# for bar in filtered_bars:
#     for year in YEARS:
#         for penetration_level in PENETRATION_LEVELS:
#             vsm_values = pd.DataFrame(pow_matrix[year][penetration_level]) \
#                            .apply(lambda x: calculate_vsm(bar, x), axis= 1)

#             save_as_csv(target_data= vsm_values,
#                         file_name= f'{bar.name}_year_{year}_pen_level_{penetration_level}.csv'.strip(),
#                         folder_path= rf'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Operative_VSMs_by_bar_{target_voltage}')


In [ ]:
fig = bar_group_heatmap(target_directory= r'Operative_VSMs_by_bar_138',
                        title= 'Operative Voltage Stability Margin Evolution <br> with Penetration Levels Over the Years for 138kV Bar Group')

fig.show()

In [ ]:
fig = bar_group_heatmap(target_directory= r'Operative_VSMs_by_bar_069',
                        title= 'Operative Voltage Stability Margin Evolution <br> with Penetration Levels Over the Years for 69kV Bar Group')

fig.show()

Housing Capacity

In [ ]:
# DATA COLLECTION DONE!!!

security_margin = 0.04
secure_PMC = PMC_CONVENTIONAL * (1 - security_margin)

for year in YEARS:
    for penetration_level in PENETRATION_LEVELS:
        avaible_power = pd.DataFrame(pow_matrix[year][penetration_level]) \
                            .apply(lambda x: secure_PMC - x)
    
        save_as_csv(target_data= avaible_power,
                    file_name= f'avaible_power_year_{year}_pen_level_{penetration_level}.csv',
                    folder_path= r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\Conventional_avaible_power')


In [ ]:
fig = housing_capacity_heatmap(target_directory= r'Conventional_avaible_power',
                               title= 'Housing Capacity Evolution <br> with Penetration Levels Over the Years'
                               )

fig.show()

In [16]:
# Filtering of bars in the target voltage groups.
selected_bars = filter_by_voltage(bars= bars, voltage_class= '069') + filter_by_voltage(bars= bars, voltage_class= '138')
power_threshold_of_selected_bars = [bar.power_threshold for bar in selected_bars]

# Dictionaries for storage.
avaible_power_dict = {}
housing_capacity_dict = {}

# Outer loop that iterates over the length of selected_bars list.
for i in range(1, len(selected_bars)+1):

    # Inner loop that iterates over the years.
    for year in YEARS:

        avaible_power_dict[year] = {}
        housing_capacity_dict[year] = {}
        
        avaible_power_list = []
        amount_cs_list = []

        # Innermost loop that repeats the avaible power calculation repeated times.
        for _ in range(1000):

            # The power threshold for normal operation limits is sampled from power_threshold_of_selected_bars list and then
            # year_load is subtracted from it. This gives an random amount of power avaible for the charging stations to be
            # stored in.   
            sampled_power_threshold = random.sample(random.sample(population= power_threshold_of_selected_bars, k= i), k= 1)[0]
            avaible_power = sampled_power_threshold - incremented_loads[YEARS.index(year)]

            # The power of the charging stations package is calculated selection three values from CHARGING_STATION_POWER.
            charging_stations = get_random_sum(loads= CHARGING_STATION_POWER, get_amount= 3, iterations= 1)
            
            # Having these values is possible to determinate the amount of charging stations that can be stored in the avaible power.
            amount_cs = avaible_power // charging_stations
            
            # All the calculated amounts are stored in a list.
            avaible_power_list.append(avaible_power)
            amount_cs_list.append(int(amount_cs))

        # The list in stored in the previously created dictionary, grouped by year.
        housing_capacity_dict[year] = amount_cs_list
        avaible_power_dict[year] = avaible_power_list

In [18]:
sorted([bar for bar in selected_bars], key= lambda bar: bar.power_threshold)

[MANACA-AM069 -- Voltage class: 069 -- Power Threshold for Operational Voltage: 2024.48,
 COMPEN-AM138 -- Voltage class: 138 -- Power Threshold for Operational Voltage: 2513.49,
 CENTRO-AM138 -- Voltage class: 138 -- Power Threshold for Operational Voltage: 2552.61,
 IRANDU-AM138 -- Voltage class: 138 -- Power Threshold for Operational Voltage: 2562.39,
 PARQ10-AM138 -- Voltage class: 138 -- Power Threshold for Operational Voltage: 2709.09,
 CGRAND-AM138 -- Voltage class: 138 -- Power Threshold for Operational Voltage: 2757.99,
 CIDNOV-AM069 -- Voltage class: 069 -- Power Threshold for Operational Voltage: 2777.55,
 IRANDU-AM069 -- Voltage class: 069 -- Power Threshold for Operational Voltage: 2982.93,
 CONSE2-AM069 -- Voltage class: 069 -- Power Threshold for Operational Voltage: 3061.17,
 MANAU2-AM069 -- Voltage class: 069 -- Power Threshold for Operational Voltage: 3061.17,
 SJOSE--AM069 -- Voltage class: 069 -- Power Threshold for Operational Voltage: 3080.73,
 PLACIB-AM069 -- Volt